# 📈 Gaggle: A Gen AI-Powered Research Assistant for Understanding Trump Tariffs

**Authors:** *Olena Mialyk*, *Yegor Mialyk*

## 🧠 Use Case

Understanding complex political-economic issues like **Trump-era tariffs** requires sifting through hours of news videos, political commentary, and dense economic analysis. This process is time-consuming and often biased or incomplete depending on the source.

### ❓ Problem

How can we help users get a **balanced**, **concise**, and **insightful understanding** of a complex political-economic topic using the power of Generative AI?

### 💬 Why We Named It *Gaggle*:

> “We called it *Gaggle* because trade policy debates are loud, messy, and multi-sourced — like a gaggle of geese.  
> *Gaggle* helps users cut through the noise to understand the real story behind Trump-era tariffs.”

### ✅ Solution with Gen AI

We built **Gaggler**, an AI-powered chatbot that aggregates and analyzes content from:
- YouTube videos
- Analytical articles from reputable news sources

Using **Google Gen AI**, we extract transcripts, search and summarize articles, embed all this information into a **vector database** (ChromaDB), and enable users to ask natural language questions and receive grounded, summarized answers with sources.

## 🚀 Gen AI Capabilities Used

| Capability                     | Description                                                      |
|-------------------------------|------------------------------------------------------------------|
| ✅ Grounding                   | Fetches and summarizes real-world articles using Google Search   |
| ✅ Embeddings & Vector Search | Stores YouTube transcripts and article content for RAG           |
| ✅ Retrieval-Augmented Generation (RAG) | Generates answers grounded in retrieved sources     |
| ✅ Structured Output           | Requests specific structured JSON from LLM                      |
| ✅ Chat with Custom Prompting  | Dynamic chatbot with reference-based response generation |

## 🔧 Code Walkthrough

### 1. 📦 Setup and Installation

Let's start from installing the Gemini API SDK, ChromaDB and other libraries.

In [1]:
!pip uninstall -qqy jupyterlab
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"
!pip install -qU youtube_transcript_api newspaper3k lxml_html_clean

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... Building wheel for pypika (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-lsp 3.10.2 requires jupyterlab<4.0.0a0,>=3.1.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.4 which is incompatible.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 5.29.4 which is incompatible.
google-spark-connect 0.5.2 requires google-api-core>=2.19.1, but you have google-api-core 1.34.1 which is

In [2]:
from google import genai
from google.genai import types

from IPython.display import Markdown, display

Set up a retry helper. This allows you to "Run all" without worrying about per-minute quota.

In [3]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

### 2. 🔑 Authentication

Your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can obtain one from [AI Studio](https://aistudio.google.com/app/apikey). Detailed instructions are available in the [documentation](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, select `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [4]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

### 3. 🎥 Extracting YouTube Video Transcripts

To build Gaggle’s semantic search and summarization pipeline entirely from real-world YouTube data — without ever downloading a video file we used `youtube-transcript-api` Python library.

To keep the workflow smooth and focused on transcript analysis, we manually specify the video URLs we want to analyze and corresponding titles.
This approach enables consistent, reproducible experiments while bypassing Kaggle’s networking limitations.

🧪 In a production or local environment, we could easily integrate `pytube` or the YouTube Data API to automate this step. Unfortunately, `pytube` is **blocked** by YouTube when running from the _Kaggle_ cloud enviroment.

In [5]:
from youtube_transcript_api import YouTubeTranscriptApi

video_uris = [
    "https://www.youtube.com/watch?v=lD3FX_wxqUI",
    "https://www.youtube.com/watch?v=iPjbj2S8S_I",
    "https://www.youtube.com/watch?v=3xz7BGAJYhg",
    "https://www.youtube.com/watch?v=KjB9pWdZL3s",
    "https://www.youtube.com/watch?v=wkKXghV2QNo",
    "https://www.youtube.com/watch?v=t1l18ILWzxw",
    "https://www.youtube.com/watch?v=rVdQC6dXlQw",
    "https://www.youtube.com/watch?v=1ts5wJ6OfzA"
]

video_titles = [  
    "Trump Tariffs Explained: The 10% Tax on All Imports",  
    "Markets skyrocket after Trump announces 90-day tariff pause",  
    "Fareed’s Take: Trump’s tariffs will bring about an orgy of corruption",  
    "Top economist predicts a recession in weeks if Trump's trade war continues",  
    "BREAKING: China retaliates against Trump tariffs, says trade war 'will become a joke'",  
    "Fox admits biz owner BLOWN UP by Trump tariffs",  
    "‘The Five’: Trump flexes his tariff muscle",  
    "Why Trump's tariff chaos actually makes sense (big picture)"  
]  

def get_video_id(url):
    if "watch?v=" in url:
        return url.split("watch?v=")[-1]
    elif "youtu.be/" in url:
        return url.split("youtu.be/")[-1]
    else:
        raise ValueError("Invalid YouTube URL")

def get_transcript(video_url):
    video_id = get_video_id(video_url)
    
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
    
    full_text = " ".join([entry['text'] for entry in transcript])
    
    return full_text

def get_transcripts(video_urls):
    transcripts = []
    
    for url in video_urls:
        transcripts.append(get_transcript(url))
        
    return transcripts

try:
    video_transcripts = get_transcripts(video_uris)
except Exception as e:
    print("Unable to get transcripts")
    video_transcripts = []
    

### 4. 🔍 Grounded Search

To complement YouTube insights, we use Google Search grounding. This setup allows Gemini to access and cite real-time web content, helping Gaggle surface reputable, analytical articles.

In [6]:
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0,
    top_p=1.0,
    top_k=40,
)

def query_with_grounding():
    
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents="""
            Use the Google Search tool to find 8 recent (published within the last 12 months), reputable analytical articles discussing the economic impact of Trump-era tariffs.
            
            Each article must be grounded in the search results and come from credible sources such as:
            - The Economist
            - Wall Street Journal (WSJ)
            - New York Times (NYT)
            - Brookings Institution
            - National Bureau of Economic Research (NBER)
            - International Monetary Fund (IMF)
            - Other academic journals or recognized economic think tanks
            
            For each article, return a JSON object in this format:
            
            {
              "title": "<The article title>",
              "url": "<Direct URL to the article>"
            }
            
            Respond with a JSON array of 8 such objects, no summaries or additional commentary. 
            Do not fabricate any URLs — only use those from the grounded search results.
            """,
        config=config_with_search
    )

    return response

response = query_with_grounding()

# print(response.candidates[0].content)

### 5. 📰 Scraping and Storing Article Content

We use the newspaper3k library to extract clean, readable article content from the URLs returned by Gemini’s grounded search. This gives Gaggle a solid foundation of web-based economic commentary to analyze alongside video transcripts.

In [7]:
import re
from newspaper import Article, Config

def scrape_full_article(link: str) -> str:

    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124  Safari/537.36'

    try:
        config = Config()
        config.browser_user_agent = user_agent

        article = Article(link, config=config)
        article.download()
        article.parse()
       
        return {"text": article.text, "title": article.title}
    except Exception as e:
        return None

content = str(response.candidates[0].content)

urls = re.findall(r'"url"\s*:\s*"([^"]+)"', content)  

documents = []
titles = []
uris = []

for url in urls:    
    
    a = scrape_full_article(url)
    if a == None: 
        continue
    
    documents.append(a["text"])    
    titles.append(a["title"])
    uris.append(url)

    print("Title:", a["title"]) 
    print("URL:", url) 
    print("========")

Title: Trump Tariffs: The Economic Impact of the Trump Trade War
URL: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAKSm-OVe5j5YfqfTypAcBgGqm6b7ZXH82GGa4FrLTdfVpcF0IniwkLvgKWxpegLZfxn4wqv9lLm0YJQdVQ_XF_H8cLeic5SDPSSDjZJagJCdisCeM1fLkI-ewcjYhT1CFggwQy380WnsncxsAww5GRHR3En4KuaxE0ghcrTXPS6UpuV
Title: Fact Sheet: President Donald J. Trump Declares National Emergency to Increase our Competitive Edge, Protect our Sovereignty, and Strengthen our National and Economic Security
URL: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAJiqKPl26B7baLjvpJ9y4yXqxhpd65k5BnNgqa_xh-fKUIahjXhy14Fhhn7PFlTprePOhkI9gk6lQcLqhYN_wmgiTttYirfqfBWgHFd9on5qI-IdF-9AL_Hm0XJwn8Pz99tEm5XZBqUEu6lgu_A6Zj3WgJwklt84lp3j_b0Xu1BdEMMk5PBMS4WTm53zzvJ3G2Wx7YD6BgeunrBtyrR14lcqVj8FA_Qn7gYI-JfyHTqLtqBy023V6CNktUEqVt5jx1_lKmr7gCifjcaUCK4yypvJvA8X9kHe26-ZXaymyS-IMAOfMsxE9qJRZJSrouUaM11do5Y6rNxIOFiVlI5ihr6FHRYbdd0-PvAj70UdeOqAs9RC7akeQ==
Title: The Economic Effects of President Trump’s Tariff

### 6. 🧠 Embeddings with Google GenAI


Enable seamless integration between Gemini's embedding model and ChromaDB

In [8]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.genai import types

class GeminiEmbeddingFunction(EmbeddingFunction):
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        
        return [e.values for e in response.embeddings]

### 7. 🗃️ Vector Database with ChromaDB

Gaggle is designed to scale. While we currently use in-memory similarity search for lightweight retrieval, we've built in compatibility with ChromaDB to support future knowledge base growth — enabling persistent storage, time-aware querying, and powerful metadata filtering across sources.


In [9]:
import chromadb

DB_NAME = "sourcesdb"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()

existing = [col for col in chroma_client.list_collections()]
if DB_NAME in existing:
    chroma_client.delete_collection(name=DB_NAME)

db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

Persist articles into ChromaDB for future semantic search and filtering.

In [10]:
db.add(documents=documents, ids=[str(i) for i in range(len(documents))],
       metadatas=[{"title": title, "url": uri, "type" : "article"} for title, uri in zip(titles, uris)])

db.count()

7

Persist videos into ChromaDB for future semantic search and filtering.

In [11]:
if video_transcripts:
    db.add(documents=video_transcripts, ids=["v" + str(i) for i in range(len(video_transcripts))],
           metadatas=[{"title": title, "url": uri, "type" : "video"} for title, uri in zip(video_titles, video_uris)])

db.count()

15

### 8. 🧾 Prompt Construction for RAG

Prepares a grounded prompt for a GenAI model using retrieved content from ChromaDB. It enables retrieval-augmented generation by embedding relevant documents in the prompt and switches embeddings to query mode for better semantic matching. 

The prompt includes references (titles + URLs) for transparency, ensuring _Gaggle's_ output is fact-based and trustworthy.

In [12]:
def get_prompt (query):
    embed_fn.document_mode = False
    
    result = db.query(query_texts=[query], include=["documents", "metadatas"])
    [all_passages] = result["documents"]
    all_metas = result["metadatas"][0]
    
    query_oneline = query.replace("\n", " ")
    
    prompt = f"""
        You are a helpful, concise, and informative assistant. Use only the reference passages below to answer the question. Respond in a complete, well-structured sentence or two, including all relevant background but omitting unrelated or unsupported information. Limit your answer to 150 words or fewer.
        
        After your answer, provide a bulleted Markdown list (unordered list) containing the titles, URLs, and [type] of only the passages you used in your answer; format as:
        - [Title](URL) [type]
        
        Do not cite passages you did not use. Return only your answer and the required list in valid Markdown.
        
        QUESTION: {query_oneline}
    """
    
    for passage, meta in zip(all_passages, all_metas):
        passage_oneline = passage.replace("\n", " ")
        
        prompt += f"TITLE: {meta['title']}\n"
        prompt += f"URL: {meta['url']}\n"
        prompt += f"TYPE: {meta['type']}\n"
        prompt += f"PASSAGE: {passage_oneline}\n"
        prompt += f"=================================\n"

    return prompt


### 9. 💬 Chat Interface

Enables an easy-to-use conversational agent with _Gaggle's_ underlying capabilities for retrieving and generating content.

In [13]:
def format_user_message(message: str) -> str:
    return f"**🧑 You:** {message}"


def format_bot_message(message: str) -> str:
    formatted_message = message.replace('\n', '\n> ')
    return f"**🦆 Gaggler:**\n\n> {formatted_message}"

def start_chat_with_prompt():
    chat = client.chats.create(model='gemini-2.0-flash', history=[])

    print("✅ Chat ready! Ask questions about the topic (type 'exit' to quit).")
    
    while True:

        Markdown(format_user_message(""))
        user_input = input()
        display(Markdown(format_user_message(user_input)))
        
        if user_input.lower() in ['exit', 'quit', 'x']:
            break

        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=get_prompt (user_input))

        display(Markdown(format_bot_message(response.text)))

start_chat_with_prompt()

✅ Chat ready! Ask questions about the topic (type 'exit' to quit).


 exit


**🧑 You:** exit

# 📚 Gaggler: Trump's Tariffs Q&A Demo

Below are the sample interactions with **Gaggler**, a Gen AI-powered research assistant, using a custom knowledge base and metadata-enabled retrieval with ChromaDB.


**🧑 You:** What are the pros and cons of Trump's tariff policy?

**🦆 Gaggler:**

> Trump's tariff policy aims to reduce trade deficits and encourage domestic production by imposing taxes on imported goods, making foreign products more expensive and incentivizing companies to invest and create jobs in the U.S. However, tariffs can raise consumer prices, reduce product variety, and disrupt global supply chains. Some economists suggest they lead to uncertainty, discourage investment, and trigger retaliation, potentially harming economic growth and reducing after-tax incomes.
> 
> - [Economist explains impact of Trump’s tariff plans](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAI2WEGAP0h8CesSooEdUBCPGUtwbEF0P_tQG2Y43v46X0QJNJRvtiB-fN_fSF9l-st0PsAtNa9b-b8ZTvc1BVhp5GeCjSN0YI4QJ0yYAc4ldgybR9RkKamUNScX7woH7eKvgAp8HPqsQ8mar-VPBMAKRELiCHbCOodbzNB2e7OScBYydU59AtcjPOKAIgdn) [article]
> - [Trump Tariffs Explained: The 10% Tax on All Imports](https://www.youtube.com/watch?v=lD3FX_wxqUI) [video]
> - [The Economic Effects of President Trump’s Tariffs — Penn Wharton Budget Model](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAKnTedLop_Q4F-7k_pqfzk84eQzpjhJFNuQadrefteupZlWq79W-jPDgO-vAqtP9cXb6XJDwFfLyAx_8w3P0KB4k8M0kErvWCwMD_vGOFw46dpXUevrzEFz0_RMCb0ao_odIBhEok6jUNI_yBtgV68_-vhuy3V5uaA5gdHsD9Zgu3WYUuimWFY32eLpMP-z59gTBEzpUu93vtYPBS70) [article]
> - [Trump Tariffs: The Economic Impact of the Trump Trade War](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAJwD_OZNiM_7uKVJat_sKTQ0nT9zedVh59mjBi0qzVSrZrgbrwuKV-tvDtz7RMQIalC3KGe88g_PPS_inc9cdSpDPrztJPzAnzXyxKb2XEor4pgUDrLi8OKnYdUebrVxQa4mJZCe2p6SdtSQiy1xYqBaJ_6XFudGiglzpAbGQvhKTM=) [article]

**🧑 You:** Compare CNN and Fox News visions on tariffs

**🦆 Gaggler:**

> Both CNN and Fox News have presented different perspectives on tariffs. Fox News aired a segment highlighting a small business owner negatively impacted by increased tariffs on imports from China, showcasing that tariffs are paid by American companies, despite the initial claim that China would bear the costs. CNN featured Republican strategist David Irvin, who said that Trump administration believes tariffs will result in more manufacturing jobs for America.
>
> - [Fox admits biz owner BLOWN UP by Trump tariffs](https://www.youtube.com/watch?v=t1l18ILWxw) [video]
> - [Top economist predicts a recession in weeks if Trump's trade war continues](https://www.youtube.com/watch?v=KjB9pWdZL3s) [video]

**🧑 You:**  What emotions do Trump’s tariffs evoke?

**🦆 Gaggler:**

> Donald Trump's tariffs have evoked sharp reactions from foreign leaders, investors, and others, while also sparking uncertainty and financial market turbulence. Some view tariffs as a way to reduce trade deficits and bring back manufacturing jobs, while others worry about increased consumer prices, reduced product variety, and potential economic slowdowns due to uncertainty and retaliation from trading partners. For some business owners, Trump’s tariffs evoke frustration at increasing costs and the possibility of shutting down.
>
> - [Economist explains impact of Trump’s tariff plans](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAI2WEGAP0h8CesSooEdUBCPGUtwbEF0P_tQG2Y43v46X0QJNJRvtiB-fN_fSF9l-st0PsAtNa9b-b8ZTvc1BVhp5GeCjSN0YI4QJ0yYAc4ldgybR9RkKamUNScX7woH7eKvgAp8HPqsQ8mar-VPBMAKRELiCHbCOodbzNB2e7OScBYydU59AtcjPOKAIgdn) [article]
> - [Fox admits biz owner BLOWN UP by Trump tariffs](https://www.youtube.com/watch?v=t1l18ILWxw) [video]
> - [Trump tariffs to knock growth but won't cause global recession, says IMF](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAIJbnHjNb6jDbAkCvQGAGzJTi2aseJ_oq2QoPiBrjp2EwjEMmPxe7ZX2MiaW6v1Tge7nCafyU1AbT4yiUCTQD2vbxVrh4jI5WyM_Kh-SwvWKwWHn1sppsxfU1X0RwX30yp_3S4qUc22CLz7jL02Y2QQSiEo1B15g_UuJ0ntXMvph0iJK3Rwhs6j1_v_3BlO_cpBIT7_oVlkhLd2aooJvcd4-1fyQg==) [article]

**🧑 You:**  How does Trump's team explain the necessity of tariffs?

**🦆 Gaggler:**

> According to the Trump administration, tariffs are a tool to reduce trade deficits, which occur when a country imports more than it exports. The logic is that tariffs make imported goods more expensive, causing imports to decline and trade to become more balanced. They also believe that tariffs will influence companies to reinvest in America, bringing jobs back to the country. Furthermore, Trump declared that foreign trade and economic practices have created a national emergency, and his order imposes responsive tariffs to strengthen the international economic position of the United States and protect American workers.
>
> - [Economist explains impact of Trump’s tariff plans](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAI2WEGAP0h8CesSooEdUBCPGUtwbEF0P_tQG2Y43v46X0QJNJRvtiB-fN_fSF9l-st0PsAtNa9b-b8ZTvc1BVhp5GeCjSN0YI4QJ0yYAc4ldgybR9RkKamUNScX7woH7eKvgAp8HPqsQ8mar-VPBMAKRELiCHbCOodbzNB2e7OScBYydU59AtcjPOKAIgdn) [article]
> - [‘The Five’: Trump flexes his tariff muscle](https://www.youtube.com/watch?v=rVdQC6dXlQw) [video]
> - [Fact Sheet: President Donald J. Trump Declares National Emergency to Increase our Competitive Edge, Protect our Sovereignty, and Strengthen our National and Economic Security](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAKeWAYJNXScC4hckZnTE2sZcp7Vw0uyCRMQ0ylTJ4FK-XZu9lO8AOfyIZKIn3ffHmnlpcVJYl4kiuhBtVl-3NA1oFVWudd-f8di8j1cs2VkPKhLeRN2yumM0B4iRo5tcVmTV-oKH6CkW-9KDuCfesiZUqM__IhSy0zXfDvI07IKuIJP7NYwGCXYAWEsShJBTuIPyUOkHQrCm4LIlJ4lD8kdlXCx7tvKxLB6Kwu93fDpUUoFTYgVMf7sIy0aa4fMusvPec58jf-qbPpt_h2oRmAc6Yd2qGUoHT8idDVHxOqtKXFEelxCBo_Qx1YOpegeDaF8CfO_yWfiZLXZlbhPaaw1gWSCQ6Ieg3ZDBPJDXM8LrfKH14Hx) [article]

**🧑 You:** What do economists say about the long-term effects of Trump’s tariffs?

**🦆 Gaggler:**

> Economists express concern about the long-term economic effects of tariffs, noting that while the direct impact may be modest, the uncertainty they create can disrupt global supply chains, rattle investor confidence, and lead companies to delay investment and hiring decisions, potentially slowing business activity and economic growth. One model projects that Trump's tariffs could reduce long-run GDP and wages significantly, leading to substantial lifetime losses for middle-income households, and could reduce GDP by 1.0 percent after accounting for foreign retaliation.
>
> - [Economist explains impact of Trump’s tariff plans](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAI2WEGAP0h8CesSooEdUBCPGUtwbEF0P_tQG2Y43v46X0QJNJRvtiB-fN_fSF9l-st0PsAtNa9b-b8ZTvc1BVhp5GeCjSN0YI4QJ0yYAc4ldgybR9RkKamUNScX7woH7eKvgAp8HPqsQ8mar-VPBMAKRELiCHbCOodbzNB2e7OScBYydU59AtcjPOKAIgdn) [article]
> - [The Economic Effects of President Trump’s Tariffs — Penn Wharton Budget Model](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAKnTedLop_Q4F-7k_pqfzk84eQzpjhJFNuQadrefteupZlWq79W-jPDgO-vAqtP9cXb6XJDwFfLyAx_8w3P0KB4k8M0kErvWCwMD_vGOFw46dpXUevrzEFz0_RMCb0ao_odIBhEok6jUNI_yBtgV68_-vhuy3V5uaA5gdHsD9Zgu3WYUuimWFY32eLpMP-z59gTBEzpUu93vtYPBS70) [article]
> - [Trump Tariffs: The Economic Impact of the Trump Trade War](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAJwD_OZNiM_7uKVJat_sKTQ0nT9zedVh59mjBi0qzVSrZrgbrwuKV-tvDtz7RMQIalC3KGe88g_PPS_inc9cdSpDPrztJPzAnzXyxKb2XEor4pgUDrLi8OKnYdUebrVxQa4mJZCe2p6SdtSQiy1xYqBaJ_6XFudGiglzpAbGQvhKTM=) [article]



# 🔖 Citation

Addison Howard and Brenda Flynn and Myles O'Neill and Nate and Polong Lin. "Gen AI Intensive Course Capstone 2025 Q1" _Kaggel_, 2025, https://kaggle.com/competitions/gen-ai-intensive-course-capstone-2025q1